In [10]:
!pip install crewai==0.120.0 crewai-tools pymupdf -q

In [ ]:
import os
from crewai import Agent, Task, Crew, Process
import getpass
import re

if "GOOGLE_API_KEY" not in os.environ:
    gemini_api_key = getpass.getpass("Enter your Google AI API key: ")
    os.environ["GOOGLE_API_KEY"] = gemini_api_key
    os.environ["GEMINI_API_KEY"] = gemini_api_key
    # os.environ["GOOGLE_API_KEY"] = gemini_api_key
    # os.environ["GEMINI_API_KEY"] = gemini_api_key


# Upload contract & define text (Handles PDF and TXT)

In [12]:
import os
import fitz

uploaded_file_path = "/content/GlobalSign_Master_Services_Agreement.pdf"
# uploaded_file_path = "/content/msa.txt"

contract_text = ""

print(f"Attempting to load contract from path: {uploaded_file_path}")

if not os.path.exists(uploaded_file_path):
    print(f"ERROR: File not found at the specified path: '{uploaded_file_path}'")

else:
    try:
        file_extension = os.path.splitext(uploaded_file_path)[1].lower()

        if file_extension == ".pdf":
            print("Detected PDF file. Attempting text extraction with PyMuPDF...")
            doc = fitz.open(uploaded_file_path)
            extracted_texts = []
            for page_num in range(len(doc)):
                page = doc.load_page(page_num)
                extracted_texts.append(page.get_text())
            doc.close()
            contract_text = "\n".join(extracted_texts) # Join text from all pages
            print(f"Successfully extracted text from {len(extracted_texts)} PDF pages.")

        elif file_extension == ".txt":
            print("Detected TXT file. Reading as plain text...")
            with open(uploaded_file_path, 'r', encoding='utf-8') as f:
                contract_text = f.read()
            print("Successfully loaded text from TXT file.")


        # Display a preview if text was extracted/read
        if contract_text:
            print("\n--- Start of Loaded/Extracted Contract Text (Preview) ---")
            print(contract_text[:1000].strip() + "...") # Show first 1000 characters
            print("--- End of Preview ---")
            if len(contract_text) < 100: # Arbitrary short length check
                 print("--- Warning: Extracted text seems very short. Check PDF content quality (e.g., scanned image?). ---")
        elif file_extension in ['.pdf', '.txt']:
             print("!!! WARNING: The file was processed but no text was extracted/read. The file might be empty, password-protected, image-based, or corrupted.")


    except Exception as e:
        print(f"!!! An error occurred while processing the file: {e}")
        contract_text = "" # Ensure text is empty on error


print("*"*70)

# Final check if contract_text is available
if not contract_text:
    print("\n!!! CRITICAL ERROR: No contract text available for analysis. File loading/extraction might have failed.")
    raise ValueError("Cannot proceed without contract text.")
else:
    print("\nContract text is ready for analysis by the Crew.")

Attempting to load contract from path: /content/GlobalSign_Master_Services_Agreement.pdf
Detected PDF file. Attempting text extraction with PyMuPDF...
Successfully extracted text from 18 PDF pages.

--- Start of Loaded/Extracted Contract Text (Preview) ---
Master Services Agreement 
THIS MASTER SERVICES AGREEMENT (THE “MSA”) AND APPLICABLE SERVICE SCHEDULE(S) GOVERN YOUR USE OF THE 
PRODUCTS AND SERVICE(S) PURCHASED BY YOU UNDER ONE OR MORE ORDER SUMMARIES. YOU MUST READ THIS MSA 
CAREFULLY BEFORE PURCHASING PRODUCTS OR SERVICES. BY CHECKING THE ACCEPTANCE BOX AND PLACING YOUR 
ORDER, YOU ARE AGREEING TO BE BOUND BY THE TERMS OF THIS MSA. IF YOU DO NOT AGREE TO THE TERMS OF THIS 
MSA, YOU WILL NOT BE PERMITTED TO ACCESS OR USE A SERVICE OR ANY PRODUCTS.   
 
BY CHECKING THE ACCEPTANCE BOX, YOU REPRESENT AND WARRANT THAT YOU ARE AUTHORIZED TO ACCEPT THIS MSA 
ON BEHALF OF THE COMPANY SHOWN IN THE ‘SOLD TO’ FIELD ON THE ORDER SUMMARY ("COMPANY" OR “YOU”) AND TO 
BIND COMPANY TO THE TERMS

# Select LLM and initialize Tools

In [13]:
# from langchain_google_genai import ChatGoogleGenerativeAI
from crewai import LLM

print("Initializing LLM...")

try:
    # llm = ChatGoogleGenerativeAI(
    #     model="gemini-2.0-flash",
    #     temperature=0,
    #     max_tokens=None,
    #     timeout=None,
    #     max_retries=2,
    # )
    llm = LLM(
        model="gemini/gemini-2.0-flash",
        temperature=0.4,
    )
    print(f"LLM ({llm.model}) initialized successfully.")
except Exception as e:
    print(f"Error initializing LLM: {e}")
    raise


Initializing LLM...
LLM (gemini/gemini-2.0-flash) initialized successfully.


# Define agents for Contract Analysis Crew

In [14]:
print("Defining Contract Analysis Agents...")

if 'llm' not in locals() or llm is None:
     raise ValueError("LLM not initialized.")

# Agent 1: Contract Parser
contract_parser = Agent(
    role='Legal Document Structure Analyst',
    goal='Segment the provided contract text into individually identifiable clauses or sections. Preserve numbering if present. Output a list where each item represents a distinct clause or paragraph.',
    backstory=(
        "You are meticulous at analyzing document structure. You identify distinct paragraphs or numbered items as separate clauses. "
        "Your goal is to break down the contract into its core components for easier analysis by other specialists. Handle potential variations in numbering or formatting."
    ),
    tools=[],
    llm=llm,
    allow_delegation=False,
    verbose=False,
    max_iter=3
)
print("- Agent 'contract_parser' defined.")

# Agent 2: Clause classifier
clause_classifier = Agent(
    role='Legal Clause Taxonomy Expert',
    goal='For each segmented clause provided, assign a likely category based on its content (e.g., Services, Term, Payment, Confidentiality, Warranty, Liability, Indemnification, Termination, Governing Law, Miscellaneous).',
    backstory=(
        "You have a deep understanding of common contract structures and the purpose of different clauses. "
        "Based on keywords and context, you can accurately categorize most standard contract provisions. "
        "Acknowledge if a clause seems miscellaneous or hard to classify."
    ),
    tools=[],
    llm=llm,
    allow_delegation=False,
    verbose=False,
    max_iter=5
)
print("- Agent 'clause_classifier' defined.")

# Agent 3: Risk Pattern Detector
risk_pattern_detector = Agent(
    role='Contract Risk Spotter',
    goal='Scan each provided contract clause text. Identify and flag clauses containing specific high-risk patterns or keywords commonly associated with unfavorable terms. Focus on patterns like: unlimited liability, broad indemnification obligations imposed ON THE CLIENT, weak warranties (e.g., extensive disclaimers), ambiguous or one-sided termination rights, automatic renewals without clear opt-out, non-standard governing law/jurisdiction choices, overly broad confidentiality obligations.',
    backstory=(
        "You are trained to recognize red flags in contract language based on common legal and business risk concerns. "
        "You meticulously compare clause text against a known set of potentially problematic phrases and concepts. "
        "You don't interpret the overall fairness, just flag the presence of specific predefined risk patterns."
    ),
    tools=[],
    llm=llm,
    allow_delegation=False,
    verbose=False,
    max_iter=7
)
print("- Agent 'risk_pattern_detector' defined.")

# Agent 4: Ambiguity Identifier
ambiguity_identifier = Agent(
    role='Clarity and Precision Analyst',
    goal='Review each contract clause for ambiguous language, undefined critical terms (e.g., "reasonable efforts", "material breach" without definition), potentially contradictory statements, or overly broad phrasing that could lead to future disputes or misinterpretations. Flag clauses requiring clarification.',
    backstory=(
        "You focus intensely on linguistic precision and clarity in legal documents. You hunt for words or phrases that lack specific definition "
        "where one might be needed, or sentences structured in a way that allows for multiple interpretations. Your aim is to flag areas needing refinement for certainty."
    ),
    tools=[],
    llm=llm,
    allow_delegation=False,
    verbose=False,
    max_iter=5
)
print("- Agent 'ambiguity_identifier' defined.")

# Agent 5: Review Brief Generator
review_brief_generator = Agent(
    role='Legal Review Summarizer',
    goal='Consolidate all flagged clauses identified by the Risk Pattern Detector and the Ambiguity Identifier into a single, structured "Review Brief". For each flagged item, clearly state the clause number (or reference), the clause text, and the specific reason(s) it was flagged (e.g., "Risk Pattern: Unlimited Liability", "Ambiguity: Undefined term \'material\'"). Organize the brief for efficient review by a legal professional.',
    backstory=(
        "You excel at synthesizing analytical findings into clear, actionable summaries for busy professionals. "
        "You organize flagged items logically, provide necessary context (clause text and reason), and ensure the output is easy to scan and understand. "
        "The goal is to facilitate a focused human review."
    ),
    tools=[],
    llm=llm,
    allow_delegation=False,
    verbose=False,
    max_iter=3
)
print("- Agent 'review_brief_generator' defined.")

print("All contract analysis agents defined.")

Defining Contract Analysis Agents...
- Agent 'contract_parser' defined.
- Agent 'clause_classifier' defined.
- Agent 'risk_pattern_detector' defined.
- Agent 'ambiguity_identifier' defined.
- Agent 'review_brief_generator' defined.
All contract analysis agents defined.


# Define Tasks for Contract Analysis Crew

In [15]:
print("Defining Contract Analysis Tasks...")

# Verify agents are defined
if 'contract_parser' not in locals() or \
   'clause_classifier' not in locals() or \
   'risk_pattern_detector' not in locals() or \
   'ambiguity_identifier' not in locals() or \
   'review_brief_generator' not in locals():
    raise ValueError("One or more required agents are not defined.")

# --- Task 1: Parse Contract into Clauses ---
task_parse_contract = Task(
    description=(
        f"Process the following contract text provided in the initial input:\n---\n{contract_text}\n---\n"
        f"Segment this text into a list of distinct clauses or sections. Attempt to preserve any existing numbering (e.g., '1.', 'Section 2.1')."
        f"Output should be a structured representation (like a numbered list or list of strings) of these segmented clauses."
    ),
    agent=contract_parser,
    expected_output=(
        "A list containing the text of each identified clause or section from the input contract. "
        "Example: ['1. SERVICES. Provider agrees...', '2. TERM. This Agreement shall...', ...]"
    )
)
print("- Task 'task_parse_contract' defined.")

# --- Task 2: Classify Clauses ---
task_classify_clauses = Task(
    description=(
        "Take the list of segmented contract clauses (provided from the previous task's context). "
        "For each clause, determine its likely primary category (e.g., Services, Term, Payment, Confidentiality, Warranty, Limitation of Liability, Indemnification, Termination, Governing Law, Entire Agreement/Miscellaneous). "
        "Present the output as a list where each item contains the original clause text and its assigned category."
    ),
    agent=clause_classifier,
    expected_output=(
        "A list, where each element corresponds to a clause and includes the clause text and its assigned category. "
        "Example: [{'clause_text': '1. SERVICES...', 'category': 'Services'}, {'clause_text': '6. LIMITATION OF LIABILITY...', 'category': 'Limitation of Liability'}, ...]"
    ),
    context=[task_parse_contract] # Needs the segmented clauses
)
print("- Task 'task_classify_clauses' defined.")

# --- Task 3: Detect Risk Patterns ---
task_detect_risks = Task(
    description=(
        "Review the list of classified contract clauses (provided in context). "
        "Scan the text of *each* clause specifically for the presence of predefined high-risk patterns or keywords (as defined in the Risk Spotter agent's goal, e.g., unlimited liability, broad client indemnification, weak warranties, ambiguous termination, auto-renewal issues, non-standard jurisdiction). "
        "Output a list containing ONLY the clauses that were flagged, including the clause text and the specific risk pattern(s) identified for each flagged clause."
    ),
    agent=risk_pattern_detector,
    expected_output=(
        "A list containing only the clauses identified as potentially risky. Each item in the list should include the clause text and the specific reason(s)/pattern(s) why it was flagged. If no risks are found, the list should be empty. "
        "Example: [{'clause_text': '6. LIMITATION OF LIABILITY...', 'risk_flag': 'Broad disclaimer of implied warranties'}, {'clause_text': '7. INDEMNIFICATION...', 'risk_flag': 'Client indemnifies Provider broadly'}]"
    ),
    context=[task_classify_clauses] # Needs the classified clauses (provides context and text)
)
print("- Task 'task_detect_risks' defined.")

# --- Task 4: Identify Ambiguities ---
task_identify_ambiguities = Task(
    description=(
        "Review the list of classified contract clauses (provided in context). "
        "Analyze the text of *each* clause for potential ambiguities, undefined critical terms, vague language, or contradictions that might require clarification. "
        "Output a list containing ONLY the clauses that were flagged for ambiguity, including the clause text and a brief explanation of the ambiguity identified in each flagged clause."
    ),
    agent=ambiguity_identifier,
    expected_output=(
        "A list containing only the clauses identified as potentially ambiguous. Each item should include the clause text and a brief note explaining the source of ambiguity. If no ambiguities are found, the list should be empty. "
        "Example: [{'clause_text': '1. SERVICES... Provider shall determine the method...', 'ambiguity_flag': 'Term \"method, details, and means\" is vague'}, {'clause_text': '8. TERMINATION... materially breaches...', 'ambiguity_flag': 'Term \"materially breaches\" is undefined'}]"
    ),
    context=[task_classify_clauses] # Needs the classified clauses
)
print("- Task 'task_identify_ambiguities' defined.")

# --- Task 5: Generate Review Brief ---
task_generate_brief = Task(
    description=(
        "Consolidate the findings from the risk detection task and the ambiguity identification task (outputs provided in context) into a structured 'Contract Review Brief'. "
        "The brief should have two main sections: 'Potential Risk Flags' and 'Potential Ambiguities/Clarifications Needed'. "
        "Under each section, list the items identified previously, including the original clause text (or a clear reference like clause number if consistently available from parsing) and the specific reason it was flagged (risk pattern or ambiguity explanation). "
        "Format the output clearly using Markdown for easy reading by a legal professional."
    ),
    agent=review_brief_generator,
    expected_output=(
        "A well-formatted Markdown document titled 'Contract Review Brief (AI Assisted)'. "
        "It should contain two distinct sections: 'Potential Risk Flags' and 'Potential Ambiguities/Clarifications Needed'. "
        "Each section should list the relevant flagged clauses with their text and the reason for flagging. "
        "The output should be ready for human review."
    ),
    # Needs the outputs from BOTH the risk detection and ambiguity identification tasks
    context=[task_detect_risks, task_identify_ambiguities]
)
print("- Task 'task_generate_brief' defined.")

print("All contract analysis tasks defined.")

Defining Contract Analysis Tasks...
- Task 'task_parse_contract' defined.
- Task 'task_classify_clauses' defined.
- Task 'task_detect_risks' defined.
- Task 'task_identify_ambiguities' defined.
- Task 'task_generate_brief' defined.
All contract analysis tasks defined.


In [16]:
print("Creating the Contract Analysis Crew...")

# Verify all components are ready
if 'contract_parser' not in locals() or \
   'clause_classifier' not in locals() or \
   'risk_pattern_detector' not in locals() or \
   'ambiguity_identifier' not in locals() or \
   'review_brief_generator' not in locals() or \
   'task_parse_contract' not in locals() or \
   'task_classify_clauses' not in locals() or \
   'task_detect_risks' not in locals() or \
   'task_identify_ambiguities' not in locals() or \
   'task_generate_brief' not in locals():
    raise ValueError("Missing agents or tasks needed to create the Contract Analysis Crew.")

# Assemble the Crew
contract_analysis_crew = Crew(
    agents=[contract_parser, clause_classifier, risk_pattern_detector, ambiguity_identifier, review_brief_generator],
    tasks=[task_parse_contract, task_classify_clauses, task_detect_risks, task_identify_ambiguities, task_generate_brief],
    process=Process.sequential,
    memory=True,
    embedder={
        "provider": "google",
        "config": {
            "model": "text-embedding-004",
            "api_key": os.environ["GOOGLE_API_KEY"]
          },
    },
    cache=True,
    verbose=False
)

print("Contract Analysis Crew created. Starting the process with kickoff()...")

crew_inputs = {
    'contract_full_text': contract_text
}

result = contract_analysis_crew.kickoff(inputs=crew_inputs)

print("CONTRACT ANALYSIS FINISHED")


Creating the Contract Analysis Crew...
Contract Analysis Crew created. Starting the process with kickoff()...


ERROR:root:Error during short_term search: 400 400 POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:embedContent?%24alt=json%3Benum-encoding%3Dint: Request payload size exceeds the limit: 36000 bytes. in query.
ERROR:root:Error during entities search: 400 400 POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:embedContent?%24alt=json%3Benum-encoding%3Dint: Request payload size exceeds the limit: 36000 bytes. in query.
ERROR:root:Error during short_term save: 400 400 POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:embedContent?%24alt=json%3Benum-encoding%3Dint: Request payload size exceeds the limit: 36000 bytes. in add.
ERROR:root:Error during short_term search: 400 400 POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:embedContent?%24alt=json%3Benum-encoding%3Dint: Request payload size exceeds the limit: 36000 bytes. in query.
ERROR:root:Error during entities searc

CONTRACT ANALYSIS FINISHED


In [17]:
if 'result' in locals():
    print(result)
else:
    print("The 'result' variable is not defined.")


```markdown
# Contract Review Brief (AI Assisted)

This document summarizes potential risks and ambiguities identified in the contract. It is intended to facilitate a focused review by a legal professional.

## Potential Risk Flags

*   **Clause 5.3 Warranty Disclaimer for Test Services:**
    *   **Clause Text:** "5.3 Warranty Disclaimer for Test Services. COMPANY ACKNOWLEDGES THAT ANY TEST SERVICES OR PRODUCTS PROVIDED \nFOR EVALUATION OR TEST PURPOSES ARE PROVIDED “AS IS” AND WITHOUT ANY WARRANTY WHATSOEVER. TO THE \nMAXIMUM EXTENT ALLOWED BY APPLICABLE LAW, GLOBALSIGN EXPRESSLY DISCLAIMS ALL REPRESENTATIONS AND \nWARRANTIES OF ANY KIND, EITHER EXPRESS OR IMPLIED, INCLUDING, WITHOUT LIMITATION, ANY WARRANTIES OF \nMERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE, OR NONINFRINGEMENT, RELATING TO THE TEST SERVICES, \nPRODUCTS, COMPANY’S USE OR ANY INABILITY TO USE A TEST SERVICE OR PRODUCTS, THE RESULTS OF THEIR USE AND \nTHIS MSA."
    *   **Reason:** Broad disclaimer of warranties 

# Contract Review Brief (AI Assisted)

This document summarizes potential risks and ambiguities identified in the contract. It is intended to facilitate a focused review by a legal professional.

## Potential Risk Flags

*   **Clause 5.3 Warranty Disclaimer for Test Services:**
    *   **Clause Text:** "5.3 Warranty Disclaimer for Test Services. COMPANY ACKNOWLEDGES THAT ANY TEST SERVICES OR PRODUCTS PROVIDED \nFOR EVALUATION OR TEST PURPOSES ARE PROVIDED “AS IS” AND WITHOUT ANY WARRANTY WHATSOEVER. TO THE \nMAXIMUM EXTENT ALLOWED BY APPLICABLE LAW, GLOBALSIGN EXPRESSLY DISCLAIMS ALL REPRESENTATIONS AND \nWARRANTIES OF ANY KIND, EITHER EXPRESS OR IMPLIED, INCLUDING, WITHOUT LIMITATION, ANY WARRANTIES OF \nMERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE, OR NONINFRINGEMENT, RELATING TO THE TEST SERVICES, \nPRODUCTS, COMPANY’S USE OR ANY INABILITY TO USE A TEST SERVICE OR PRODUCTS, THE RESULTS OF THEIR USE AND \nTHIS MSA."
    *   **Reason:** Broad disclaimer of warranties for Test Services, including merchantability and fitness for a particular purpose.

*   **Clause 5.4 LIMITATION OF LIABILITY FOR TEST SERVICES:**
    *   **Clause Text:** "5.4 LIMITATION OF LIABILITY FOR TEST SERVICES. GLOBALSIGN SHALL NOT BE LIABLE TO COMPANY OR ANY THIRD PARTY \nFOR ANY CLAIMS, DEMANDS OR DAMAGES WHATSOEVER, INCLUDING, WITHOUT LIMITATION, DIRECT, INDIRECT, \nCONSEQUENTIAL OR SPECIAL DAMAGES, ARISING OUT OF THE USE OF THE PRODUCTS OR TEST SERVICES FOR EVALUATION \nOR TEST PURPOSES AND THE USE OR FAILURE OF THE TEST SERVICES TO OPERATE FOR WHATEVER REASON, WHETHER \nSUCH ACTION IS BASED IN CONTRACT OR TORT OR OTHERWISE, INCLUDING, WITHOUT LIMITATION, NEGLIGENCE."
    *   **Reason:** Broad limitation of liability for Test Services, including direct, indirect, consequential, or special damages, even in cases of negligence.

*   **Clause 9.5 No Other Warranty:**
    *   **Clause Text:** "9.5 No Other Warranty.  EXCEPT AS PROVIDED IN THE CPS OR TPS  AT https://www.globalsign.com/en/repository, AND TO \nTHE MAXIMUM EXTENT ALLOWED BY APPLICABLE LAW, GLOBALSIGN, ITS AFFILIATES, AND THEIR RESPECTIVE SUCCESSORS, \nDIRECTORS, OFFICERS, EMPLOYEES, AND AGENTS DISCLAIM ALL OTHER WARRANTIES AS TO THE USE, DELIVERY, LICENSE, \nPERFORMANCE OR NONPERFORMANCE OF, USE OR INABILITY TO USE THE SERVICES, PRODUCTS, THIRD PARTY PRODUCTS, \nCERTIFICATES, SOFTWARE, DOCUMENTATION OR ANY OTHER SERVICES OFFERED OR CONTEMPLATED BY THIS \nAGREEMENT, EXPRESS OR IMPLIED. GLOBALSIGN, ITS AFFILIATES, AND THEIR RESPECTIVE SUCCESSORS, DIRECTORS, \nOFFICERS, EMPLOYEES, AND AGENTS EXPRESSLY DISCLAIM ALL REPRESENTATIONS AND WARRANTIES OF ANY KIND, EITHER \nEXPRESS OR IMPLIED, INCLUDING, WITHOUT LIMITATION, ANY WARRANTIES OF MERCHANTABILITY, FITNESS FOR A \nPARTICULAR PURPOSE, OR NONINFRINGEMENT. GLOBALSIGN DOES NOT WARRANT THAT THE SERVICE OR ANY PRODUCTS \nWILL BE UNINTERRUPTED OR ERROR-FREE."
    *   **Reason:** Broad disclaimer of warranties, including merchantability, fitness for a particular purpose, and uninterrupted/error-free service.

*   **Clause 12.3 Indemnification:**
    *   **Clause Text:** "12.3 Company will settle and/or defend at its own expense and indemnify and hold harmless GlobalSign against any cost, \nloss or damage from any claim, demand, suit or action brought by a third party against GlobalSign arising out of or related \nto any (i) breach of this MSA by Company, (ii) use of the Service or Products by a third party who is accessing or acquiring \nthe Service or Products through Company, (iii) use of any Third Party Products in combination with the Service, or (iv)  \nCompany’s failure to comply with Section 2.1 above."
    *   **Reason:** Client (Company) indemnifies Provider (GlobalSign) against broad claims, including those arising from third-party use of services accessed through the client, use of third-party products, and breach of MSA.

## Potential Ambiguities/Clarifications Needed

*   **Clause 3 Limitations on Use:**
    *   **Clause Text:** "3. Limitations on Use. Company shall not: (e) use the Services other than in accordance with this MSA and in compliance with all applicable Industry Standards, laws and regulations.  In addition to any specific use limitations \nthat may be set forth in a Service Schedule, GlobalSign may set and enforce limits for reasonable use in order to prevent \nabuse of or undue burden on the Services."
    *   **Reason:** The term "reasonable use" is not defined, leading to potential disputes about what constitutes abuse or undue burden.

*   **Clause 7.1 Termination (Material Breach):**
    *   **Clause Text:** "7.1 Termination. By GlobalSign: This MSA or any Service Schedule may be terminated by \nGlobalSign upon written notice to Company: (a) if Company materially breaches this MSA or any Service Schedule and such \nbreach continues for a period of thirty (30) days after notice thereof has been given by GlobalSign"
    *   **Reason:** The term "materially breaches" is undefined, making it unclear what level of breach warrants termination.

*   **Clause 7.1 Termination (Security/Compliance Risk):**
    *   **Clause Text:** "7.1 Termination. Notwithstanding the above, this MSA may be terminated immediately by GlobalSign upon \nwritten notice if GlobalSign determines, in its reasonable discretion, that Company poses a security or compliance risk to \nthe Service or GlobalSign."
    *   **Reason:** The phrase "reasonable discretion" is subjective and could lead to disputes regarding what constitutes a legitimate security or compliance risk. The terms "security risk" and "compliance risk" are also undefined.

*   **Clause 10 Confidentiality:**
    *   **Clause Text:** "10. Confidentiality. “Confidential Information” means all information that is provided or made available to one party (the \n“Receiving Party”) by the other party (the “Disclosing Party”). Confidential Information includes, but is not limited to: \ninventions, technologies; strategies; trade secrets; customer and supplier lists; product designs and pricing information; \nprocesses; formulas; business plans; employer and consumer information; employee data; product licensing plans; \nbudgets, finances, and financial plans; production plans and protocols; technology infrastructure; information security \nsystems, policies and practices, and technology, data, and methods, and any other information that by its nature would \ntypically be considered non-public information."
    *   **Reason:** The phrase "any other information that by its nature would typically be considered non-public information" is vague and subjective. It lacks a clear standard for determining what qualifies as confidential.

*   **Confidentiality - Degree of Care:**
    *   **Clause Text:** "The Receiving Party will protect, and will ensure its employees, officers, agents and contractors will protect Confidential \nInformation by using the same degree of care as Receiving Party uses to protect its own Confidential Information of a like \nnature (but no less than a reasonable degree of care) to prevent the unauthorized use, dissemination, disclosure or \npublication of such Confidential Information."
    *   **Reason:** The phrase "reasonable degree of care" is subjective and lacks a clear standard. It's unclear what actions would satisfy this requirement.

*   **Clause 12.2 Infringement Claim Resolution:**
    *   **Clause Text:** "12.2 Should the Services become, or in GlobalSign’s sole opinion likely to become, the subject of any claim or action for \ninfringement, GlobalSign may (a) procure, at no cost to Company, the right for Company to continue using the Services as \ncontemplated hereunder; (b) modify the Service, without loss of material functionality or performance, to render the \nServices non-infringing; or (c) if the foregoing alternatives are not reasonably available to GlobalSign, terminate this MSA."
    *   **Reason:** The phrase "reasonably available" is subjective and could lead to disputes regarding what constitutes reasonable availability. It's unclear what factors GlobalSign should consider when determining if the alternatives are reasonably available.
